In [ ]:
import numpy as np
import pyvista as pv
import vedo as vd

from pathlib import Path

vd.settings.set_vtk_verbosity(0)

# Functions

In [ ]:
def convert_unstructured_to_polydata_mesh(input_mesh):
    point_data = input_mesh.point_data
    cell_data = input_mesh.cell_data

    polydata_mesh = pv.PolyData.from_regular_faces(
        input_mesh.points, input_mesh.cells.reshape(-1, 4)[:, 1:]
    )

    for key in point_data.keys():
        polydata_mesh.point_data[key] = point_data[key]
    for key in cell_data.keys():
        polydata_mesh.cell_data[key] = cell_data[key]

    return polydata_mesh


def coarsen_mesh(input_mesh, decimation_factor):
    point_data = input_mesh.point_data
    for data in input_mesh.point_data:
        del input_mesh.point_data[data]

    mesh_with_point_data = input_mesh.cell_data_to_point_data()
    coarsened_mesh_with_point_data = mesh_with_point_data.decimate(
        decimation_factor, volume_preservation=False, scalars=True, vectors=True
    )
    coarsened_mesh = coarsened_mesh_with_point_data.point_data_to_cell_data()
    for data in point_data:
        coarsened_mesh.point_data[data] = point_data[data]

    return coarsened_mesh


def smoothen_region_boundaries(
    input_mesh,
    smoothing_strategy=0,
    num_iterations=50,
    relaxation_factor=0.1,
):
    point_data_mesh = input_mesh.cell_data_to_point_data()
    vd_mesh = vd.Mesh([point_data_mesh.points, input_mesh.faces.reshape(-1, 4)[:, 1:]])
    vd_mesh.pointdata["anatomical_tags"] = point_data_mesh.point_data["anatomical_tags"]
    vd_mesh = vd_mesh.smooth_data(
        niter=num_iterations, relaxation_factor=relaxation_factor, strategy=smoothing_strategy
    )
    point_data_mesh.point_data["anatomical_tags"] = vd_mesh.pointdata["anatomical_tags"]
    smoothened_mesh = point_data_mesh.point_data_to_cell_data()

    return smoothened_mesh


def fix_tags_after_interpolation(
    input_mesh, region_labels, assignment_threshold, lower_cutoff=0.1
):
    max_label = max(region_labels.values())
    feature_meshes = input_mesh.extract_values(
        ranges=[lower_cutoff, max_label + 1e-5], scalars="anatomical_tags"
    )
    separated_meshes = feature_meshes.split_bodies()

    for label in separated_meshes.keys():
        mesh_portion = separated_meshes[label]
        mesh_portion = extract_tags_above_threshold(mesh_portion, assignment_threshold)
        input_mesh.cell_data["anatomical_tags"][
            mesh_portion.cell_data["vtkOriginalCellIds"]
        ] = mesh_portion.cell_data["anatomical_tags"]
    near_zero_tags = np.where(input_mesh.cell_data["anatomical_tags"] <= lower_cutoff)[0]
    input_mesh.cell_data["anatomical_tags"][near_zero_tags] = 0
    input_mesh = fill_boundary_dents(input_mesh)
    input_mesh = fill_boundary_spikes(input_mesh)

    return input_mesh


def extract_tags_above_threshold(input_mesh_region, assignment_threshold):
    maximum_tag_value = np.max(input_mesh_region.cell_data["anatomical_tags"])
    threshold_value = assignment_threshold * maximum_tag_value
    feature_cell_inds = np.where(
        input_mesh_region.cell_data["anatomical_tags"] >= threshold_value
    )[0]
    body_cell_inds = np.setdiff1d(
        np.arange(input_mesh_region.n_cells), feature_cell_inds
    )
    input_mesh_region.cell_data["anatomical_tags"][feature_cell_inds] = (
        maximum_tag_value
    )
    input_mesh_region.cell_data["anatomical_tags"][body_cell_inds] = 0
    return input_mesh_region


def fill_boundary_dents(input_mesh):
    body_cell_inds = np.where(input_mesh.cell_data["anatomical_tags"] == 0)[0]
    for cell_ind in body_cell_inds:
        cell_neighbors = input_mesh.cell_neighbors(cell_ind, connections="edges")
        neighbor_tags = input_mesh.cell_data["anatomical_tags"][cell_neighbors]
        non_body_tags = neighbor_tags[neighbor_tags != 0]
        if np.size(non_body_tags) > 1:
            input_mesh.cell_data["anatomical_tags"][cell_ind] = non_body_tags[0]

    return input_mesh

def fill_boundary_spikes(input_mesh):
    feature_cell_inds = np.where(input_mesh.cell_data["anatomical_tags"] != 0)[0]
    for cell_ind in feature_cell_inds:
        cell_neighbors = input_mesh.cell_neighbors(cell_ind, connections="edges")
        neighbor_tags = input_mesh.cell_data["anatomical_tags"][cell_neighbors]
        body_tags = neighbor_tags[neighbor_tags == 0]
        if np.size(body_tags) > 1:
            input_mesh.cell_data["anatomical_tags"][cell_ind] = 0

    return input_mesh

# Load Dataset

In [ ]:
patient_id = "patient_01"
infile_name = Path(f"../data/raw/{patient_id}_mesh_with_uacs_fibers_tags.vtk")
outfile_name = Path(f"../data/processed/{patient_id}/mesh_with_fibers_tags.vtk")
legacy_anatomical_tags = {
    "Body": 11,
    "LAA": 13,
    "LIPV": 21,
    "LSPV": 23,
    "RIPV": 25,
    "RSPV": 27,
}

new_anatomical_tags = {
    "Body": 0,
    "LAA": 1,
    "LIPV": 2,
    "LSPV": 3,
    "RIPV": 4,
    "RSPV": 5,
}

input_mesh = pv.read(infile_name)
triangular_mesh = convert_unstructured_to_polydata_mesh(input_mesh)
del triangular_mesh.point_data["alpha"]
del triangular_mesh.point_data["beta"]
for key, value in legacy_anatomical_tags.items():
    anatomical_region = np.where(triangular_mesh.cell_data["anatomical_tags"] == value)[
        0
    ]
    triangular_mesh.cell_data["anatomical_tags"][anatomical_region] = (
        new_anatomical_tags[key]
    )
triangular_mesh

# Coarsen Mesh for faster computations

In [ ]:
coarse_mesh = coarsen_mesh(triangular_mesh, decimation_factor=0.8)
coarse_mesh = fix_tags_after_interpolation(
    coarse_mesh, new_anatomical_tags, assignment_threshold=0.4
)
np.unique(coarse_mesh.cell_data["anatomical_tags"])

In [ ]:
outfile_name.parent.mkdir(parents=True, exist_ok=True)
pv.save_meshio(outfile_name, coarse_mesh)

In [ ]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    coarse_mesh,
    show_edges=True,
    scalars="anatomical_tags",
    cmap="tab20",
    show_scalar_bar=False,
    edge_color="lightgray",
    edge_opacity=0.3,
)
plotter.show()